# Лабораторная работа №2. Обработка датасета

В прошлой работе мы познакомились с консольными командами и менеджерами для установки различного функционала в проект. Начиная с этой лабораторной, мы плавно погружаемся в процесс обучения нейронных сетей.

За любым методом машинного обучения стоит некоторая статистика, а за любой статистикой стоят некоторые оцениваемые объекты. Они могут иметь разные характеристики, могут подлежать различной обработке, но без них не достичь никакого результата.

**Сразу запоминаем несколько важных условий, серьезно влияющих на применение статистики:**

1. Статистики из одного объекта не бывает.
1. Статистика по двум объектам составляется только из их прямой зависимости друг от друга => нельзя оценить разнообразие.
1. Объекты, кардинально отличающиеся от остальных, могут препятствовать корректности общей статистики.
1. Если объекты можно условно разделить на несколько независимых категорий, то зависимость по ним будет строится, исходя из средних значений их признаков (т.н. центров масс множеств).
1. При нестандартной (нелинейной) зависимости объектов друг от друга необходимо соблюдать закон, по которому они представлены.

На изображении ниже вы можете видеть визуализацию каждого случая, представленного в списке.

<img src="https://raw.githubusercontent.com/703lovelost/HomeworksDC/refs/heads/main/src/lab2/statistic_plots.png">

Уточним еще один важный момент: представим, что существуют многомерные представления объектов, не имеющие за собой математической закономерности, но для которых использовался некоторый ассоциативный ряд. К таким представлениям относятся изображения и аудиозаписи. Человек может явно описать их содержание (например, увидеть собаку на фотографии или услышать шум автомобиля в аудиозаписи). Однако если разложить изображение на пиксели, либо аудиозапись на фрагменты цифрового сигнала, то определить действительные закономерности с помощью методов ML становится очень сложно.

<img src="https://wallpapers.com/images/high/puppy-pictures-umqvikc7wzgxzc9w.webp">

Последующие открытия в данном вопросе привели к появлению Deep Learning - методов глубокого машинного обучения, позволяющих эффективно рассматривать контекст данных и обобщать их для решения конкретных задач, таких как:

* Классификация - определение принадлежности объекта к одному/нескольким рассматриваемым наименованиям;
* Сегментация - разметка (н-ер, пикселей изображений) в соответствии с рассматриваемыми наименованиями;
* Детекция - определение зон нахождения объектов, включая их классификацию.

## Задание 1.

Настало время вспомнить опыт предыдущей работы и загрузить свой первый датасет для исследования.

Для поиска датасетов используется сервис Kaggle. Именно с него мы возьмем наш первый [датасет, в котором представлены изображения 10 различных видов цветов.](https://www.kaggle.com/datasets/aksha05/flower-image-dataset) Скачаем и разархивируем его.

После каждой запущенной ячейки не забывайте обновлять вкладку _Files_ в меню слева, либо запускать аналогичную команду `cd`.

In [ ]:
# Все, что начинается после знака штрихкода - это комментарий к коду/скрипту в ячейке.
# cURL - прекрасный способ выгрузить любую информацию из интернета, в том числе скачать необходимые вам файлы.
# Так можно выгружать даже веб-страницы. Попробуйте как-нибудь выгрузить HTML-разметку главной страницы Google (google.com).

!curl -L -o ./flower-image-dataset.zip https://www.kaggle.com/api/v1/datasets/download/aksha05/flower-image-dataset

In [ ]:
# unzip - классический пакет Linux для разархивирования файлов. Есть и другие, но он стандартный и легко устанавливается.

!unzip -q ./flower-image-dataset.zip

## Задание 2.

У вас появилась папка _flowers_. Откроем ее и увидим дикую кучу файлов.

В любом датасете важна правильная организация файлов, с которой вы получите ряд преимуществ:

* Значительно упростится навигация по файлам.
* Появится возможность управлять отдельными рядами файлов - например, обогащать отдельные классы, об этом чуть позже.
* Снизится к нулю вероятность "утечки данных", когда объекты в ходе обучения модели были применены не по назначению.

Мы видим, что в скачанном датасете цветы _хотя бы подписаны_ (еще такие подписи называются лейблами, дальше я буду называть их так). Это нам уже значительно упрощает упорядочивание.

Сейчас мы с вами займемся сортировкой файлов по папкам в соответствии с их лейблами. Подготовим наш код. Запустите ячейку ниже.

In [ ]:
# Импорт библиотек Python для использования в коде.

from __future__ import annotations

import shutil
from pathlib import Path
from typing import Dict, Iterable, List, Tuple, Optional

# Константы - постоянные значения.

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tif", ".tiff", ".webp"}

# Немного функций общего пользования.

def is_image(path: Path) -> bool:
    return path.is_file() and path.suffix.lower() in IMG_EXTS and not path.name.startswith("._")

def extract_label_from_filename(fname: str) -> str:
    stem = Path(fname).stem
    return stem.rsplit("_", 1)[0] if "_" in stem else stem

def ensure_empty_dir(p: Path, force: bool = False) -> None:
    if p.exists():
        if force:
            shutil.rmtree(p)
        else:
            raise FileExistsError(f"Папка уже существует: {p}. Укажите force=True для перезаписи.")
    p.mkdir(parents=True, exist_ok=True)

def uniquify(dst: Path) -> Path:
    if not dst.exists():
        return dst
    stem, ext = dst.stem, dst.suffix
    i = 1
    while True:
        cand = dst.with_name(f"{stem}__dup{i}{ext}")
        if not cand.exists():
            return cand
        i += 1

__Важно: если в проекте Google Colab ранее была запущена ячейка с импортами, то она будет применена ко всем остальным ячейкам.
Более того, все ячейки сохраняют контекст ранее запущенных ячеек.__

__Номер запуска ячейки вы можете найти в квадратных скобках слева от любой исполняемой ячейки.__

При запуске ячеек пока еще ничего не происходит, кроме подгрузки в среду библиотек, констант и функций.

Ниже идет сама функция для преобразования папки _flowers_ в нужный нам формат. Запустите ячейку.

In [ ]:
def classify_flowers(
    src_dir: Path | str = "flowers",
    out_dir: Path | str = "flowers_classified",
    *,
    force: bool = False,
    return_manifest: bool = True,
):
    src_dir = Path(src_dir)
    out_dir = Path(out_dir)

    if not src_dir.exists():
        raise FileNotFoundError(f"Исходная папка не найдена: {src_dir}")

    ensure_empty_dir(out_dir, force=force)

    files = [p for p in src_dir.rglob("*") if is_image(p)]
    if not files:
        raise RuntimeError(f"В {src_dir} не найдено изображений с расширениями: {sorted(IMG_EXTS)}")

    per_class_counts: Dict[str, int] = {}
    manifest: List[Dict[str, str]] = []

    for f in files:
        label = extract_label_from_filename(f.name)
        class_dir = out_dir / label
        class_dir.mkdir(parents=True, exist_ok=True)

        dst = uniquify(class_dir / f.name)
        shutil.copy2(f, dst)

        per_class_counts[label] = per_class_counts.get(label, 0) + 1
        if return_manifest:
            manifest.append({"src": str(f), "dst": str(dst), "label": label})

    # Краткая сводка
    total = sum(per_class_counts.values())
    n_classes = len(per_class_counts)
    print(f"Всего файлов: {total} | Классов: {n_classes}")
    for k in sorted(per_class_counts):
        print(f"  {k}: {per_class_counts[k]}")

    return manifest if return_manifest else None

Теперь перейдем к главному зрелищу. Ячейка ниже позволяет, наконец, запустить процесс сортировки. Не томите, запускайте :)

In [ ]:
manifest = classify_flowers(src_dir="./flowers", out_dir="./flowers_classified", force=True, return_manifest=True)

По результатам вывода мы можем видеть, сколько изображений представлено для каждого вида - они все достаточно представлены, датасет хорошо сбалансирован.

Также мы можем непосредственно доказать, что там действительно 10 видов (классов). Это нам и требовалось.

Результат можно также посмотреть в ваших файлах в папке _flowers_classified_. Согласитесь, другое дело.

## Задание 3.

Теперь настало время для серьезного разговора.

Данные для обучения делятся на три типа:

* Обучающие данные (training data) - выборка из датасета для знакомства модели с объектами и для выделения свойств. Модель калибруется под выделение тех или иных признаков в объектах для дальнейшей работы с ними.
* Валидационные данные (validation data) - выборка из датасета для уточнения качества обучения модели. При прохождении валидационных данных через модель она уже считается откалиброванной, оператор может сам подобрать дополнительные параметры для улучшения обработки моделью.
* Тестовые данные (testing data) - данные для проверки итоговой версии модели, не имеющие никакого влияния на ее функциональность. Редко представлены в самом датасете, поскольку нужны либо для непосредственного обучения, либо донастройки.

Изображение из датасета может быть только в одной из вышеописанных выборок. Это главное условие для избежания "утечки данных".

Сейчас мы с вами будем делить датасет на составляющие его выборки, а именно training и validation.

Существует два способа это сделать:

* Виртуально разделить датасет на подвыборки при подгрузки датасета для обучения (как это делает, например, [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html))
* Сразу разделить датасет на соответствующие подпапки.

Со вторым пунктом идеально справляется библиотека [splitfolders](https://pypi.org/project/split-folders/). Установим ее.

In [ ]:
!pip install split-folders

Используя splitfolders, вы сможете наяву посмотреть деление датасета и увидеть всю красоту. Нам в этом также поможет соответствующая функция.

In [ ]:
def split_flowers(
    classified_dir: Path | str = "flowers_classified",
    out_split_dir: Path | str = "flowers_split",
    *,
    train_ratio: float = 0.85,
    val_ratio: float = 0.15,
    seed: int = 42,
    force: bool = False,
    move: bool = False,
):
    try:
        import splitfolders  # pip install split-folders
    except ImportError as e:
        raise SystemExit("Не найден пакет splitfolders. Установите: pip install split-folders") from e

    classified_dir = Path(classified_dir)
    out_split_dir = Path(out_split_dir)

    if not classified_dir.exists():
        raise FileNotFoundError(f"Папка с классами не найдена: {classified_dir}")

    if abs((train_ratio + val_ratio) - 1.0) > 1e-6:
        raise ValueError(f"Сумма долей должна быть 1.0, сейчас: {train_ratio + val_ratio}")

    ensure_empty_dir(out_split_dir, force=force)

    splitfolders.ratio(
        input=str(classified_dir),
        output=str(out_split_dir),
        seed=seed,
        ratio=(train_ratio, val_ratio),
        move=move,
    )
    print(f"\ntrain/val = {train_ratio:.2f}/{val_ratio:.2f} → {out_split_dir}")

Соотношения обучающих данных к валидационных подбираются такие, чтобы обучающие значительно превалировали по числу. Обычно берется соотношение 85/15, в особенности, если датасет маленький, как этот.

Ну что ж, вперед запускать и смотреть результат у себя в файлах!

In [ ]:
split_flowers("./flowers_classified", "./flowers_split", train_ratio=0.85, val_ratio=0.15, force=True)

## Задание 4.

Несложно заметить, что есть признаки, которые объединяют определенные виды цветов - как минимум, их расцветки уже могут сбить модель с толку. Не забываем также о том, что датасет в целом маленький, и надо бы как-то его обогатить.

Для этого существует такая практика, как [аугментация данных](https://habr.com/ru/companies/smartengines/articles/264677/) - преобразование изображений датасета таким образом, чтобы модель могла лучше выделить одни качества объекта над другими, например, форму над цветом.

Принцип работы таков:

1. Объявляются методы преобразования картинок перед отправкой на обучение модели. Прописываются вероятности, с которыми те или иные преобразования будут применены.
1. Модель обучается, хватая картинки небольшими пачками (т.н. батчами). С некоторой прописанной вероятностью в батч вместо оригинальной картинки будет направлена ее преобразованная версия.

Здесь мы рассмотрим такую библиотеку, как [albumentations](https://pypi.org/project/albumentations/) - огромный и удобный в использовании каталог аугментаций для изображений.

In [ ]:
!pip install albumentations

Объявим все необходимое для запуска аугментаций. Замечу, что в данном примере все аугментации будут иметь вероятность в 100%, чтобы вы точно не пропустили ни одну из них.

In [55]:
from __future__ import annotations
import random
from pathlib import Path
from typing import List, Dict

import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as A


# Немного общих функций.

def list_images(folder: str | Path) -> List[Path]:
    folder = Path(folder)
    if not folder.exists():
        raise FileNotFoundError(f"Папка не найдена: {folder}")
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

def imread_rgb(path: Path) -> np.ndarray:
    img_bgr = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise ValueError(f"Не удалось прочитать изображение: {path}")
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)

# Сами аугментации.

def build_aug_transforms(img_size: int = 224) -> Dict[str, A.BasicTransform]:
    return {
        "RandomResizedCrop":  A.RandomResizedCrop(size=(img_size, img_size), scale=(0.75, 1.0), ratio=(0.9, 1.1), p=1.0),
        "HorizontalFlip":     A.HorizontalFlip(p=1.0),
        "ToGray":             A.ToGray(method='weighted_average', p=1.0),
        "HueSaturationValue": A.HueSaturationValue(
            hue_shift_limit=[-40, 20], sat_shift_limit=[0, 90], val_shift_limit=[-20, 20], p=1.0
        ),
        "CoarseDropout":      A.CoarseDropout(
            hole_height_range=[0.5, img_size // 8],
            hole_width_range=[0.5, img_size // 8],
            num_holes_range=[1, 4],
            fill=0,
            p=1.0
        ),
    }

def apply_transforms(img_rgb: np.ndarray, transforms: Dict[str, A.BasicTransform]) -> Dict[str, np.ndarray]:
    out = {}
    for name, t in transforms.items():
        aug = A.Compose([t])
        out[name] = aug(image=img_rgb)["image"]
    return out

# Визуализация.

def show_augmentations_grid(
    image_paths: List[Path],
    transforms: Dict[str, A.BasicTransform],
    img_size: int = 224,
    max_samples: int = 3,
    seed: int = 42,
):
    set_seed(seed)
    if len(image_paths) == 0:
        raise RuntimeError("Список изображений пуст.")
    sample_paths = random.sample(image_paths, k=min(max_samples, len(image_paths)))

    n_aug = len(transforms)
    n_cols = n_aug + 1
    n_rows = len(sample_paths)

    plt.figure(figsize=(3.0 * n_cols, 3.2 * n_rows))

    for r, path in enumerate(sample_paths):
        img = imread_rgb(path)
        base = A.Compose([A.LongestMaxSize(max_size=img_size), A.PadIfNeeded(img_size, img_size, border_mode=cv2.BORDER_CONSTANT, value=(0,0,0))])(image=img)["image"]

        ax = plt.subplot(n_rows, n_cols, r * n_cols + 1)
        ax.imshow(base)
        ax.set_title("Original", fontsize=11)
        ax.axis("off")

        aug_images = apply_transforms(base, transforms)
        for c, (name, aug_img) in enumerate(aug_images.items(), start=2):
            ax = plt.subplot(n_rows, n_cols, r * n_cols + c)
            ax.imshow(aug_img)
            ax.set_title(name, fontsize=11)
            ax.axis("off")

    plt.tight_layout()
    plt.show()

Настало время для итоговой генерации. Запускайте ячейку!

In [ ]:
train_images = list_images("./flowers_split/train")

augs = build_aug_transforms(img_size=224)
show_augmentations_grid(train_images, augs, img_size=224, max_samples=10, seed=42)

## Задание 5.

Разработчики albumentations подготовили сайт, где вы можете попробовать онлайн каждую из представленных аугментаций.

[https://explore.albumentations.ai/](https://explore.albumentations.ai/)

Ознакомьтесь с аугментациями на сайте.

При сдаче работы преподаватель даст вам определенную тему датасета. Будьте готовы назвать 2-3 аугментации, которые вы бы применили, чтобы обогатить датасет и улучшить качество модели по различию классов в датасете.